# 享元模式

[参考链接]
- :star: :star: :star: http://yloiseau.net/articles/DesignPatterns/flyweight/
- https://github.com/faif/python-patterns/blob/master/structural/flyweight.py

享元模式(Flyweight Pattern)：运用共享技术有效地支持大量细粒度对象的复用。系统只使用少量的对象，而这些对象都很相似，状态变化很小，可以实现对象的多次复用。由于享元模式要求能够共享的对象必须是细粒度对象，因此它又称为轻量级模式，它是一种对象结构型模式。

享元模式以共享的方式高效地支持大量细粒度对象的重用，享元对象能做到共享的关键是区分了内部状态(Intrinsic State)和外部状态(Extrinsic State)：
- 内部状态是存储在享元对象内部并且不会随环境改变而改变的状态，内部状态可以共享。如字符的内容，不会随外部环境的变化而变化，无论在任何环境下字符“a”始终是“a”，都不会变成“b”。
- 外部状态是随环境改变而改变的、不可以共享的状态。享元对象的外部状态通常由客户端保存，并在享元对象被创建之后，需要使用的时候再传入到享元对象内部。

## 通过工厂模式实现

In [58]:
class Spam(object):
    def __init__(self, a, b):
        self.a = a
        self.b = b


class SpamFactory(object):
    def __init__(self):
        self.__instances = dict()

    def get_instances(self, a, b):
        key = ','.join(map(str, [a, b]))
        if key not in self.__instances:
            self.__instances[key] = Spam(a, b)
        return self.__instances[key]


spam_factory = SpamFactory()

In [64]:
spam1 = spam_factory.get_instances(1, 2)
spam2 = spam_factory.get_instances(1, 2)
spam3 = spam_factory.get_instances(1, 3)
print spam1 is spam2
print spam1 is not spam3

True
True


## 通过抽象工厂模式实现

In [71]:
class FlyweightFactory(object):
    def __init__(self, kls):
        self._kls = kls
        self._instances = dict()

    def get_instance(self, *args, **kwargs):
        key = ','.join(map(str, args) + [str(kwargs)])
        if key not in self._instances:
            self._instances[key] = self._kls(*args, **kwargs)
        return self._instances[key]


class Foo(object):
    def __init__(self, a, b):
        self.a = a
        self.b = b


class Bar(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y


foo_factory = FlyweightFactory(Foo)
bar_factory = FlyweightFactory(Bar)

print foo_factory.get_instance(1, 2) is foo_factory.get_instance(1, 2)
print foo_factory.get_instance(1, 2) is not foo_factory.get_instance(1, 3)
print bar_factory.get_instance(1, 2) is not foo_factory.get_instance(1, 2)

True
True
True


## 通过类装饰器实现

In [44]:
class flyweight(object):
    def __init__(self, kls):
        print 'flyweight __init__', kls.__name__
        self._kls = kls
        self._instances = dict()

    def __call__(self, *args, **kwargs):
        print 'flyweight __call__'
        key = ','.join(map(str, args) + [str(kwargs)])
        if key not in self._instances:
            self._instances[key] = self._kls(*args, **kwargs)
        return self._instances[key]


@flyweight
class Foo(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y


@flyweight
class Bar(object):
    def __init__(self, a, b):
        self.a = a
        self.b = b

flyweight __init__ Foo
flyweight __init__ Bar


In [45]:
print isinstance(Foo, flyweight)

foo1 = Foo(1, 2)
foo2 = Foo(1, 2)
foo3 = Foo(1, 3)
bar1 = Bar(1, 2)

True
flyweight __call__
flyweight __call__
flyweight __call__
flyweight __call__


In [46]:
print foo1 is foo2
print foo2 is not foo3
print foo1 is not bar1

True
True
True


## 享元模式: 重构`__new__`

In [16]:
import weakref


class Card(object):
    _card_pool = weakref.WeakValueDictionary()

    def __new__(cls, value, suit):
        key = str(value) + str(suit)
        card = cls._card_pool.get(key)
        if not card:
            card = object.__new__(cls)
            cls._card_pool[key] = card
            card.value, card.suit = value, suit
        return card

    def __repr__(self):
        return '<Card: %s-%s>' % (self.value, self.suit)

In [17]:
card1 = Card(6, 'diamond')
card2 = Card(6, 'diamond')

print card1, card2
print card1 == card2
print card1 is card2

<Card: 6-diamond> <Card: 6-diamond>
True
True


In [18]:
card1.tmp = 'aaa'
print card2.tmp

card3 = Card(6, 'diamond')
print card3.tmp

aaa
aaa


## 享元模式: 通过元类

In [15]:
import weakref


class FlyweightMeta(type):
    def __new__(cls, name, parents, dct):
        """
        :param name: class name
        :param parents: class parents
        :param dct: dict contains class attrs, methods
        :return: a new class instance
        """
        print 'FlyweightMeta __new__'
        dct['pool'] = weakref.WeakValueDictionary()
        # 1 or 3 arguments is need when create a `type` object
        return super(FlyweightMeta, cls).__new__(cls, name, parents, dct)

    @staticmethod
    def _serialize_params(cls, *args, **kwargs):
        print 'FlyweightMeta _serialize_params'
        args = list(map(str, args))
        args += [str(kwargs), cls.__name__]
        return ''.join(args)

    def __call__(cls, *args, **kwargs):
        print 'FlyweightMeta __call__'
        pool = getattr(cls, 'pool', {})
        key = FlyweightMeta._serialize_params(cls, *args, **kwargs)
        obj = pool.get(key)
        if obj is None:
            obj = super(FlyweightMeta, cls).__call__(*args, **kwargs)
            pool[key] = obj
        return obj

In [36]:
def _metaclass(meta, *bases):
    """兼容python2/3"""
    return meta("flyweight", bases, {})


class FlyCard(_metaclass(FlyweightMeta)):
    def __init__(self, *args, **kwargs):
        pass

FlyweightMeta __new__
FlyweightMeta __new__


In [17]:
fly_card1 = FlyCard('6', 'heart', 1)
print '=' * 20
fly_card2 = FlyCard('6', 'heart', 1)
print '=' * 20
fly_card3 = FlyCard('6', 'heart', '2')

FlyweightMeta __call__
FlyweightMeta _serialize_params
FlyweightMeta __call__
FlyweightMeta _serialize_params
FlyweightMeta __call__
FlyweightMeta _serialize_params


In [18]:
print fly_card1 is fly_card2
print fly_card1 is not fly_card3
print len(FlyCard.pool) == 2

True
True
True


## 享元模式: 通过Mixin

In [32]:
import weakref


class FlyWeightMixin(object):
    pool = weakref.WeakValueDictionary()

    def __new__(cls, *args, **kwargs):
        items = list(map(str, args))
        items += [str(kwargs), cls.__name__]
        key = ','.join(items)
        if key not in cls.pool:
            obj = super(FlyWeightMixin, cls).__new__(cls, *args, **kwargs)
            cls.pool[key] = obj
        return cls.pool[key]


class Foo(FlyWeightMixin):
    def __init__(self, a, b):
        self.a = a
        self.b = b


class SubFoo(Foo):
    pass

In [34]:
foo1 = Foo(1, 2)
foo2 = Foo(1, 2)
subf = SubFoo(1, 2)

print foo1 is foo2
print foo1 is not subf

True
True
